In [73]:
# default_exp transplier

# longtail

> API details.

In [3]:
#export
import os
from dateutil import parser
from collections import namedtuple

with BlockTracker("../data/00000000.txt",stanza="Mid file 2") as f:
	f(print)("hello")
	f(print)("world")
	f(capture)("hello","world")
	f(print)("human fish")
	f(waste_some_time)(100)
with BlockTracker("../data/00000000.txt",stanza="Mid file 2") as f:
	f(print)("block 2")
	f(waste_some_time)(300)


In [ ]:
#export

Blameline = namedtuple("Blameline","hash date func sig line")

In [85]:
#export

class TransplileGroup:

    def updateMeta(self,meta):
        self.meta = meta
        if "name" in meta:
            self.name = meta["name"].strip() + f"_{self.counter}"



    def __init__(self,  meta,counter):
        self.lines = []
        self.updateMeta(meta)
        self.counter = counter
        if "name" in meta:
            self.name = meta["name"].strip() + f"_{self.counter}"
        else:
            self.name = f"myfunc_{counter}"

    def addLine(self,line:Blameline):
        self.lines.append(line)


    def postProcess(self, writer):
        #print("in the post",group,len(group))
        dates = [x.date for x in self.lines]

        if len(self.lines)>0:
            # using max() + lambda
            # to get tuple info. of maximum value tuple
            res = max(self.lines, key = lambda i : i[1])

            # printing result
            #print ("The name with maximum score is : " + res)
            writer.write(f"\ndef {self.name}():\n")
            writer.write(f"\twith BlockTracker(\"data/{res.hash}.txt\",stanza=\"Mid file 2\") as f:\n")
            for x in self.lines:
                writer.write(f"\t\tf({x.func})({x.sig})\n")



In [89]:
#export



class Transpiler(object):



    def postMeta(self,line):
        line = line[1:]
        parts = line.split(":")
        if len(parts) > 1:
            self.meta[parts[0].strip()] = ":".join(parts[1:]).rstrip()


    def parseBlame(self,line):
        st = line.index(" (")
        et = line.index(") ")

        hash = line[:st]
        label = line[st+2:et-1]
        prog = line[et+2:]

        timestamp = " ".join(label.split(" ")[-4:])
        dt_obj = parser.parse(timestamp)

        if prog[0] == "@":
            self.postMeta(prog)
            return None, False

        items = prog.split("\t")
        if len(items) < 2:
            return None , True

        func = items[0]
        sig = items[1]
        sig = sig[:-1]

        return Blameline(hash=hash,date=timestamp,func=func,sig = sig,line = line), False

    def parse(self):
        groups = []
        cmd = 'cd {path};git blame {fname}'.format(
                    path=self.root,
                    fname=self.file_name)
        with os.popen(cmd) as process:
            blame =  process.readlines()


        with  open(self.outfile, 'w') as writer:


            functionCount = 0
            code_group = TransplileGroup(self.meta,functionCount)
            for x in blame:
                fields, isEnd = self.parseBlame(x)
                if isEnd:
                    code_group.updateMeta(self.meta)
                    if len(code_group.lines) > 0:
                        functionCount += 1
                        groups.append(code_group)
                        code_group = TransplileGroup(self.meta,functionCount)
                elif  fields is not None:
                    code_group.addLine(fields)

            if len(code_group.lines) > 0:
                code_group.updateMeta(self.meta)
                groups.append(code_group)

            writer.write(f"# AUTOGENERATED! DO NOT EDIT! File to edit:{self.file_name} (unless otherwise specified).\n")
            glist = "\", \"".join([x.name for x in groups])
            writer.write(f"__all__ = [\"{glist}\"]\n\n")

            writer.write("""
from longtail.engine import BlockTracker
from longtail.commonlib import waste_some_time, lastWrapper, capture

""")
            for group in groups:
                group.postProcess(writer)


            writer.write("\n\nif __name__ == \"__main__\":\n")
            for group in groups:
                writer.write(f"\t{group.name}()\n")
                print(group.name)


    def __init__(self,  root,file_name,out_file):
        self.file_name = file_name
        self.root = root
        self.outfile = out_file
        self.meta = {}


In [90]:
#hide
tr = Transpiler(".","test/src/basic.txt","robogen/basic.py")
tr.parse()

myfunc_0
deepQuerry_1


In [87]:
#hide
!cat robogen/basic.py

# AUTOGENERATED! DO NOT EDIT! File to edit:test/src/basic.txt (unless otherwise specified).
__all__ = ["myfunc_0", "deepQuerry_1"]


from longtail.engine import BlockTracker
from longtail.commonlib import waste_some_time, lastWrapper, capture


def myfunc_0():
	with BlockTracker("data/ab568dce.txt",stanza="Mid file 2") as f:
		f(print)("hello")
		f(print)("world")
		f(capture)("hello","world")
		f(print)("human fish")
		f(waste_some_time)(100)

def deepQuerry_1():
	with BlockTracker("data/ab568dce.txt",stanza="Mid file 2") as f:
		f(print)("block 2")
		f(waste_some_time)(300)


if __name__ == ‘__main__’:
	myfunc_0()
	deepQuerry_1()


In [62]:
from robogen.basic import deepQuerry_1


ImportError: cannot import name 'deepQuerry_1' from 'robogen.basic' (/Users/arthurconner/code/midtail/robogen/basic.py)

hello
world
hello
world
human fish


In [91]:
#hide
!python robogen/basic.py

hello
world
hello
world
human fish
block 2


hello
world
human fish


,name,signature,value,start,total,batch,stanza
0,capture,"'hello', 'world'",None,4860.93,0.000340,4860.931309,Mid file 2
1,print,'human fish',None,4860.93,0.000021,4860.931309,Mid file 2
2,waste_some_time,100,None,4860.93,0.264946,4860.931309,Mid file 2
